# Convolutional Neural Networks
- IMDB review sentiment classification with CNN
  - Last time, we have started sentence classification with CNN having only one filter. Here, let's try more complicated CNN architecture with different filter sizes to ameliorate the performance.

In [0]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 591.8MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61d2e000 @  0x7f29476992a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 18.0MB/s 
    100% |████████████████████████████████| 2.0MB 4.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [1]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.0.0'

## 1. Import & process dataset
- IMDB review dataset for sentiment analysis
  - [source](http://ai.stanford.edu/~amaas/data/sentiment/)
  - Let's cheat a while and use dataset provided by Keras

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence

num_words = 10000
maxlen = 50

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

X_train = sequence.pad_sequences(X_train, maxlen = maxlen, padding = 'pre')
X_test = sequence.pad_sequences(X_test, maxlen = maxlen, padding = 'pre')
    
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Using TensorFlow backend.


(25000, 50) (25000, 50) (25000,) (25000,)


## 2. Creating CNN model and training

- Create and train CNN model for sentence classification, with three convolutional & max pooling layers concatenated in the end.
- Model architecture is adopted from [Kim 2015](https://www.aclweb.org/anthology/D14-1181)

![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/08/Example-of-a-CNN-Filter-and-Polling-Architecture-for-Natural-Language-Processing.png)

In [0]:
class imdbTrainDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.X = X_train
    self.y = y_train
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
  
  def __len__(self):
    return len(self.X)
  
class imdbTestDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.X = X_test
    self.y = y_test
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
  
  def __len__(self):
    return len(self.X)

In [0]:
# create dataset & dataloader instances
train_dataset = imdbTrainDataset()
test_dataset = imdbTestDataset()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128, shuffle = False)

In [0]:
# create CNN with one convolution/pooling layer
class net(nn.Module):
  def __init__(self, input_dim, num_words, embedding_dim, num_filters, kernel_size, stride):
    super(net, self).__init__()
    self.input_dim = input_dim
    self.embedding_dim = embedding_dim
    
    conv_output_size1 = int((input_dim - kernel_size[0])/stride) + 1   # first conv layer output size
    conv_output_size2 = int((input_dim - kernel_size[1])/stride) + 1   # first conv layer output size
    conv_output_size3 = int((input_dim - kernel_size[2])/stride) + 1   # first conv layer output size
        
    self.embedding = nn.Embedding(num_words, self.embedding_dim)
    
    # three convolution & pooling layers
    self.conv1 = nn.Conv2d(1, num_filters[0], kernel_size = (kernel_size[0], self.embedding_dim), stride = stride)     
    self.pool1 = nn.MaxPool2d((conv_output_size1, 1))                # Max-over-time pooling
    self.conv2 = nn.Conv2d(1, num_filters[1], kernel_size = (kernel_size[1], self.embedding_dim), stride = stride)     
    self.pool2 = nn.MaxPool2d((conv_output_size2, 1))                # Max-over-time pooling
    self.conv3 = nn.Conv2d(1, num_filters[2], kernel_size = (kernel_size[2], self.embedding_dim), stride = stride)     
    self.pool3 = nn.MaxPool2d((conv_output_size3, 1))                # Max-over-time pooling
    
    self.relu = nn.ReLU()
    self.dense = nn.Linear(num_filters[0] + num_filters[1] + num_filters[2], 2)     
    
  def forward(self, x):
    x = self.embedding(x)                                   # project to word embedding space
    x = x.view(-1, 1, self.input_dim, self.embedding_dim)   # resize to fit into convolutional layer
    x1 = self.pool1(self.relu(self.conv1(x)))
    x2 = self.pool2(self.relu(self.conv2(x)))
    x3 = self.pool3(self.relu(self.conv3(x)))

    x = torch.cat((x1, x2, x3), dim = 1)   # concatenate three convolutional outputs
    x = x.view(x.size(0), -1)   # resize to fit into final dense layer
    x = self.dense(x)
    return x

In [0]:
# hyperparameters
DEVICE = torch.device('cuda')
INPUT_DIM = maxlen
NUM_FILTERS = (16, 32, 64) 
KERNEL_SIZE = (1, 2, 3)
STRIDE = 1
EMBEDDING_DIM = 50
NUM_WORDS = num_words
LEARNING_RATE = 1e-3
NUM_EPOCHS = 30         

In [0]:
model = net(INPUT_DIM, NUM_WORDS, EMBEDDING_DIM, NUM_FILTERS, KERNEL_SIZE, STRIDE).to(DEVICE)
criterion = nn.CrossEntropyLoss()   # do not need softmax layer when using CEloss criterion
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [27]:
# training for NUM_EPOCHS
for i in range(NUM_EPOCHS):
  temp_loss = []
  for (x, y) in train_loader:
    x, y = x.long().to(DEVICE), y.to(DEVICE)  # beware that input to embedding should be type 'long'
    outputs = model(x)
    loss = criterion(outputs, y)
    temp_loss.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print("Loss at {}th epoch: {}".format(i, np.mean(temp_loss)))

Loss at 0th epoch: 0.6226605290965158
Loss at 1th epoch: 0.4895881914666721
Loss at 2th epoch: 0.40360755549401656
Loss at 3th epoch: 0.3413624836474049
Loss at 4th epoch: 0.2855290376714298
Loss at 5th epoch: 0.23665862393622494
Loss at 6th epoch: 0.19332292257827155
Loss at 7th epoch: 0.1526499463587391
Loss at 8th epoch: 0.11854718325241488
Loss at 9th epoch: 0.09007945675783012
Loss at 10th epoch: 0.06637739711346066
Loss at 11th epoch: 0.04888947682493195
Loss at 12th epoch: 0.03602837147761365
Loss at 13th epoch: 0.026454947622759
Loss at 14th epoch: 0.019765267642785092
Loss at 15th epoch: 0.015103718741055655
Loss at 16th epoch: 0.011833845148319188
Loss at 17th epoch: 0.00931674547075313
Loss at 18th epoch: 0.007469219165112899
Loss at 19th epoch: 0.006082932326980695
Loss at 20th epoch: 0.005003999198824927
Loss at 21th epoch: 0.004151264236460687
Loss at 22th epoch: 0.0034760919789194452
Loss at 23th epoch: 0.0029148718803094662
Loss at 24th epoch: 0.002471668743148294
Loss 

## 3. Evaluation
- Evaluate the trained CNN model with accuracy score 
  - Store probability of each instance to a list and compare it with true y label

In [28]:
y_pred, y_true = [], []
with torch.no_grad():
  for x, y in test_loader:
    x, y = x.long().to(DEVICE), y.to(DEVICE)       # beware that input to embedding should be type 'long'
    outputs = F.softmax(model(x)).max(1)[-1]       # predicted label
    y_true += list(y.cpu().numpy())                # true label
    y_pred += list(outputs.cpu().numpy())   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [29]:
# evaluation result
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.80096